In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os
import shutil
from sklearn.model_selection import train_test_split
from random import shuffle 
from tqdm import tqdm 
from PIL import Image
import warnings
warnings.filterwarnings('ignore')


# The images were placed in four folders depending on if they were for testing
# or training and if they contained a bat. The images were split 15% testing
# and 85% training
train_bat = "bat_train"
train_no_bat= "no_bat_train"

In [ ]:
# All images were resized to 150 X 150. This can be changed depending on how much RAM is available
image_size = 150

# Converts training images to numpy arrays and preprocesses them
def train_data():
    train_data_bat = [] 
    train_data_no_bat=[]
    for image1 in tqdm(os.listdir(train_bat)): 
        path = os.path.join(train_bat, image1)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        train_data_bat.append(img1) 
    for image2 in tqdm(os.listdir(train_no_bat)): 
        path = os.path.join(train_no_bat, image2)
        img2 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img2 = cv2.resize(img2, (image_size, image_size))
        train_data_no_bat.append(img2) 
    
    train_data= np.concatenate((np.asarray(train_data_bat),np.asarray(train_data_no_bat)),axis=0)
    return train_data

train_data = train_data()
print("Processed training data")

 96%|████████████████████████████████████████████████████████████████████████▏  | 16107/16722 [00:58<00:02, 299.90it/s]

In [3]:
x_data = train_data
x_data = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))

In [5]:
o1 = np.ones(16722)
z1 = np.zeros(15010)
Y_train = np.concatenate((o1, z1), axis=0)

In [ ]:
y_data=np.asarray(Y_train).reshape(x_data.shape[0],1)
print("X shape: " , x_data.shape)
print("Y shape: " , y_data.shape)

In [ ]:
# Split data

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=42)
number_of_train = x_train.shape[0]
number_of_test = x_test.shape[0]